# ทำนายผลผลิตข้าวโพด โดยใช้ผลผลิตปีก่อน ๆ ด้วย SARIMAX


In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

register_matplotlib_converters()

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

Get data

In [10]:
yield_file = 'OAE-process/OAE-ผลผลิตข้าวโพดทั้งหมด.xlsx'
df = pd.read_excel(yield_file)

areas = df.groupby(['area']).sum().sort_values(by='value', ascending=False)

In [11]:
def data_in_area(df, area):
    df = df.sort_values(by='date', ascending=True)
    df_area = df[df.area == area].reset_index()
    df_area = df_area[['date', 'value']]
    df_area.columns = ['ds', 'y']
    return df_area

In [12]:
def resample_year2month(df):
    df = df.set_index('date')
    df.index.name = 'ds'
    return df.resample('MS').asfreq().fillna(method='ffill')

In [13]:
cost_file = 'OAE-process/OAE-ต้นทุนรวมต่อไร่ข้าวโพดเลี้ยงสัตว์.xlsx'
df_cost = pd.read_excel(cost_file)
df_cost_spl = resample_year2month(df_cost)

precipitation_file = 'OAE-process/OAE-ปริมาณน้ำฝน-ฝนตก.xlsx'
df_precipitation = pd.read_excel(precipitation_file)
df_precipitation_nan = data_in_area(df_precipitation,'Nan').set_index('ds')

rainday_file = 'OAE-process/OAE-ปริมาณน้ำฝน-จำนวนวันฝนตก.xlsx'
df_rainday = pd.read_excel(rainday_file)
df_rainday_nan = data_in_area(df_rainday,'Nan').set_index('ds')

price_file = 'OAE-process/OAE-ราคาข้าวโพดเลี้ยงสัตว์.xlsx'
df_price = pd.read_excel(price_file)
df_price_spl = df_price.set_index('date')

Util functions

In [14]:
def add_features(df):
    df = df.join(df_cost_spl['value'], on='ds').rename(columns={'value':'cost'})
    df = df.join(df_precipitation_nan['y'].rename('precipitation'), on='ds')
    df = df.join(df_rainday_nan['y'].rename('rainday'), on='ds')
    df = df.join(df_price_spl['value'], on='ds').rename(columns={'value':'price'})
    return df.sort_values(by='ds', ascending=False).fillna(method='bfill')

def is_harvest_season(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 10 and date.month <= 12)

# Dashboard

In [15]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [28]:
@widgets.interact_manual(Province=areas.index[:30])
def predict(Province=areas.index[0]):

    df_yield = data_in_area(df, Province)
    print(f'Total Yield from {df.date.min().strftime("%b %Y")} to {df.date.max().strftime("%b %Y")} = {areas.loc[Province,"value"]}')
    
    df_yield = add_features(df_yield)
    df_yield['on_season'] = df_yield['ds'].apply(is_harvest_season).astype('float')
    df_yield = df_yield.sort_values(by='ds', ascending=True)

    df_yield = df_yield.set_index('ds')
    df_yield.index = pd.DatetimeIndex(df_yield.index)
    df_yield = df_yield.asfreq(pd.infer_freq(df_yield.index))
    
    df_train, df_test = df_yield[:-12], df_yield[-12:]
    
    df_yield['year'] = pd.DatetimeIndex(df_yield.index).year
    print(df_yield.groupby('year').sum().head())
    
    endog = df_train.loc[:, 'y']
    exog = df_train.loc[:, ['cost', 'precipitation', 'rainday', 'price']]
    
    my_order = (1,0,1)
    my_seasonal_order = (0, 1, 0, 12)
    # define model
    model = SARIMAX(endog=endog, exog=exog, order=my_order, seasonal_order=my_seasonal_order)
    #fit the model
    model_fit = model.fit()
    print(model_fit.summary())
    
    # exog_forecast = sm.add_constant(df_test.loc[:, ['cost', 'precipitation', 'rainday', 'price', 'on_season']])
    exog_forecast = df_test.loc[:, ['cost', 'precipitation', 'rainday', 'price']]
    #get the predictions and residuals
    predictions = model_fit.forecast(len(df_test), exog=exog_forecast)
    # predictions = pd.Series(predictions, index=df_test.index)
    predictions.index = df_test.index
    predictions.clip(lower=0, inplace=True)
    residuals = df_test['y'] - predictions
    
    print('MAE: %f' % np.mean(np.abs(residuals)) )
    print('Mean Absolute Percent Error:', round(np.mean(abs(residuals / df_test['y'])),4))
    print('Root Mean Squared Error:', np.sqrt(np.mean(residuals**2)))
    
    total_error = abs(1 - (np.sum(predictions) / np.sum(df_test['y']))) * 100
    print('Total Error: %.3f' % total_error)
    print('Forecast:', np.sum(predictions))
    print('Ground Truth:', np.sum(df_test['y']))
    
    plt.figure(figsize=(10,4))
    plt.plot(residuals)
    plt.axhline(0, linestyle='--', color='k')
    plt.title('Residuals from SARIMA Model', fontsize=20)
    plt.ylabel('Error', fontsize=16)
    
    plt.figure(figsize=(10,4))
    plt.plot(df_yield['y'])
    plt.plot(predictions)
    plt.legend(('Data', 'Predictions'), fontsize=16)
    
    print(predictions.head())
    
    py.iplot([
        go.Scatter(x=df_train.index, y=df_train['y'], name='train', line=dict(width=3)),
        go.Scatter(x=predictions.index, y=predictions, name='yhat', line=dict(width=3)),
#         go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
#         go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
#         go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend'),
        go.Scatter(x=df_test.index, y=df_test['y'], name='test', marker=dict(color='blue', size=12), line=dict(width=3)),
    ])
    
#     py.iplot(plot_plotly(model, forecast))

#     py.iplot(plot_components_plotly(model, forecast))    

interactive(children=(Dropdown(description='Province', options=('Phetchabun', 'Nakhon Ratchasima', 'Nan', 'Tak…